In [37]:
from pprint import pprint
assets = ['USDC', 'USDT']

In [38]:
from FetchKucoin import Kucoin
kucoin = Kucoin()
kucoin.addAssets(assets=assets)
kucoin.getSimpleEarnRates()

In [39]:
pprint(kucoin.SimpleEarnRates)

{'USDC': [{'amt': '', 'rate': '3.7%'}], 'USDT': [{'amt': '', 'rate': '7.6%'}]}
